Initialization of ES

In [1]:
from elasticsearch import Elasticsearch

client = Elasticsearch(['host.docker.internal'])
indice = "syslog-2021-02-24,syslog-2021-02-25"

1. Query All 

In [2]:
query = {
    "query": {
        "match_all": {}
    }
}

res = client.search(index=indice, body=query)

print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(timestamp)s %(raw)s" % hit["_source"])

Got 2000 Hits:
2021-02-24T23:44:56.160Z <117>2 2021-02-24T23:44:56.160Z random.net ahmadajmi 4895 ID3 - Pretty pretty pretty good
2021-02-24T23:44:56.170Z <62>1 2021-02-24T23:44:56.170Z make.de devankoshal 9927 ID51 - You're not gonna believe what just happened
2021-02-24T23:44:56.180Z <163>2 2021-02-24T23:44:56.180Z names.org devankoshal 8168 ID903 - Great Scott! We're never gonna reach 88 mph with the flux capacitor in its current state!
2021-02-24T23:44:56.190Z <58>2 2021-02-24T23:44:56.190Z we.net Karimmove 7105 ID300 - Great Scott! We're never gonna reach 88 mph with the flux capacitor in its current state!
2021-02-24T23:44:56.200Z <29>1 2021-02-24T23:44:56.200Z up.org shaneIxD 2747 ID774 - There's a breach in the warp core, captain
2021-02-24T23:44:56.210Z <106>2 2021-02-24T23:44:56.210Z for.com benefritz 8575 ID884 - We're gonna need a bigger boat
2021-02-24T23:44:56.221Z <83>1 2021-02-24T23:44:56.221Z some.org jesseddy 7273 ID869 - A bug was encountered but not in Vector, which

2. Match Query

In [3]:
query = {
    "query": {
        "match": {
            "hostname":"for.org"
        }
    }
}

res = client.search(index=indice, body=query)

print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(timestamp)s %(raw)s" % hit["_source"])

Got 55 Hits:
2021-02-24T23:45:55.605Z <72>2 2021-02-24T23:45:55.605Z for.org Karimmove 7152 ID40 - #hugops to everyone who has to deal with this
2021-02-24T23:45:56.405Z <60>1 2021-02-24T23:45:56.405Z for.org devankoshal 8081 ID57 - There's a breach in the warp core, captain
2021-02-24T23:45:56.775Z <153>1 2021-02-24T23:45:56.775Z for.org ahmadajmi 5366 ID570 - Pretty pretty pretty good
2021-02-24T23:45:58.605Z <53>2 2021-02-24T23:45:58.605Z for.org ahmadajmi 3743 ID567 - Great Scott! We're never gonna reach 88 mph with the flux capacitor in its current state!
2021-02-24T23:45:59.366Z <166>2 2021-02-24T23:45:59.366Z for.org ahmadajmi 7944 ID524 - Pretty pretty pretty good
2021-02-24T23:45:59.675Z <136>2 2021-02-24T23:45:59.675Z for.org meln1ks 7736 ID839 - Great Scott! We're never gonna reach 88 mph with the flux capacitor in its current state!
2021-02-24T23:45:59.775Z <185>2 2021-02-24T23:45:59.775Z for.org jesseddy 2285 ID424 - Pretty pretty pretty good
2021-02-24T23:45:59.785Z <153>

3. Multi Match

In [41]:
query = {
    "query": {
        "multi_match": {
            "query": "up.com ahmadajmi", 
            "fields":["hostname", "application"]
        }
    }
}

res = client.search(index=indice, body=query)

print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(timestamp)s %(raw)s" % hit["_source"])

Got 344 Hits:
2021-02-25T00:06:38.719Z <141>1 2021-02-25T00:06:38.719Z up.com shaneIxD 4366 ID379 - There's a breach in the warp core, captain
2021-02-25T00:06:39.089Z <82>2 2021-02-25T00:06:39.089Z up.com shaneIxD 2355 ID672 - Great Scott! We're never gonna reach 88 mph with the flux capacitor in its current state!
2021-02-25T00:06:39.180Z <27>2 2021-02-25T00:06:39.180Z up.com devankoshal 6642 ID699 - Great Scott! We're never gonna reach 88 mph with the flux capacitor in its current state!
2021-02-25T00:06:39.359Z <94>2 2021-02-25T00:06:39.359Z up.com jesseddy 1305 ID842 - #hugops to everyone who has to deal with this
2021-02-25T00:06:40.198Z <179>2 2021-02-25T00:06:40.198Z up.com ahmadajmi 3459 ID731 - Take a breath, let it go, walk away
2021-02-25T00:06:40.280Z <115>2 2021-02-25T00:06:40.280Z up.com benefritz 7890 ID688 - #hugops to everyone who has to deal with this
2021-02-25T00:06:40.909Z <29>2 2021-02-25T00:06:40.909Z up.com shaneIxD 5866 ID573 - You're not gonna believe what ju

4. String Query

In [42]:
query = {
  "query": {
    "query_string": {
      "query": "(for.org) AND (pretty breath) "
    }
  }
}

res = client.search(index=indice, body=query)

print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(timestamp)s %(raw)s" % hit["_source"])

Got 18 Hits:
2021-02-24T23:45:56.775Z <153>1 2021-02-24T23:45:56.775Z for.org ahmadajmi 5366 ID570 - Pretty pretty pretty good
2021-02-24T23:45:59.366Z <166>2 2021-02-24T23:45:59.366Z for.org ahmadajmi 7944 ID524 - Pretty pretty pretty good
2021-02-24T23:45:59.775Z <185>2 2021-02-24T23:45:59.775Z for.org jesseddy 2285 ID424 - Pretty pretty pretty good
2021-02-24T23:46:02.795Z <84>2 2021-02-24T23:46:02.795Z for.org Karimmove 505 ID463 - Pretty pretty pretty good
2021-02-24T23:46:04.095Z <159>2 2021-02-24T23:46:04.095Z for.org meln1ks 2157 ID267 - Pretty pretty pretty good
2021-02-25T00:06:40.220Z <31>2 2021-02-25T00:06:40.220Z for.org Karimmove 3849 ID886 - Pretty pretty pretty good
2021-02-25T00:06:43.269Z <180>1 2021-02-25T00:06:43.269Z for.org jesseddy 8296 ID270 - Pretty pretty pretty good
2021-02-25T00:06:44.079Z <35>2 2021-02-25T00:06:44.079Z for.org devankoshal 9479 ID338 - Pretty pretty pretty good
2021-02-25T00:06:44.259Z <22>1 2021-02-25T00:06:44.259Z for.org benefritz 8207 ID

5. Term Query

In [43]:
query = {
   "query":{
      "term":{"message":"pretty"}
   }
}

res = client.search(index=indice, body=query)

print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(timestamp)s %(raw)s" % hit["_source"])

Got 204 Hits:
2021-02-25T00:06:38.789Z <105>2 2021-02-25T00:06:38.789Z make.net devankoshal 2805 ID223 - Pretty pretty pretty good
2021-02-25T00:06:38.809Z <179>2 2021-02-25T00:06:38.809Z names.net jesseddy 5290 ID681 - Pretty pretty pretty good
2021-02-25T00:06:39.459Z <13>1 2021-02-25T00:06:39.459Z make.net shaneIxD 9737 ID15 - Pretty pretty pretty good
2021-02-25T00:06:39.519Z <67>2 2021-02-25T00:06:39.519Z make.net benefritz 3783 ID716 - Pretty pretty pretty good
2021-02-25T00:06:39.579Z <59>1 2021-02-25T00:06:39.579Z random.de devankoshal 5210 ID119 - Pretty pretty pretty good
2021-02-25T00:06:39.619Z <67>2 2021-02-25T00:06:39.619Z for.de meln1ks 5203 ID213 - Pretty pretty pretty good
2021-02-25T00:06:39.679Z <65>1 2021-02-25T00:06:39.679Z up.org shaneIxD 7571 ID545 - Pretty pretty pretty good
2021-02-25T00:06:39.719Z <79>1 2021-02-25T00:06:39.719Z make.com meln1ks 1331 ID968 - Pretty pretty pretty good
2021-02-25T00:06:39.759Z <3>1 2021-02-25T00:06:39.759Z make.org meln1ks 9407 I

6. Range Query

In [48]:
query = {
   "query":{
      "range":{
         "version":{
            "gte":2
         }
      }
   }
}

res = client.search(index=indice, body=query)

print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(timestamp)s %(raw)s" % hit["_source"])

Got 968 Hits:
2021-02-24T23:44:56.160Z <117>2 2021-02-24T23:44:56.160Z random.net ahmadajmi 4895 ID3 - Pretty pretty pretty good
2021-02-24T23:44:56.180Z <163>2 2021-02-24T23:44:56.180Z names.org devankoshal 8168 ID903 - Great Scott! We're never gonna reach 88 mph with the flux capacitor in its current state!
2021-02-24T23:44:56.190Z <58>2 2021-02-24T23:44:56.190Z we.net Karimmove 7105 ID300 - Great Scott! We're never gonna reach 88 mph with the flux capacitor in its current state!
2021-02-24T23:44:56.210Z <106>2 2021-02-24T23:44:56.210Z for.com benefritz 8575 ID884 - We're gonna need a bigger boat
2021-02-24T23:45:55.325Z <106>2 2021-02-24T23:45:55.325Z some.us benefritz 9677 ID477 - Maybe we just shouldn't use computers
2021-02-24T23:45:55.366Z <1>2 2021-02-24T23:45:55.366Z names.net benefritz 2088 ID187 - A bug was encountered but not in Vector, which doesn't have bugs
2021-02-24T23:45:55.395Z <49>2 2021-02-24T23:45:55.395Z up.de benefritz 8523 ID509 - You're not gonna believe what 

7. Exist Query

In [49]:
query = {
  "query": {
    "exists": {
      "field": "application"
    }
  }
}

res = client.search(index=indice, body=query)

print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(timestamp)s %(raw)s" % hit["_source"])

Got 2000 Hits:
2021-02-24T23:44:56.160Z <117>2 2021-02-24T23:44:56.160Z random.net ahmadajmi 4895 ID3 - Pretty pretty pretty good
2021-02-24T23:44:56.170Z <62>1 2021-02-24T23:44:56.170Z make.de devankoshal 9927 ID51 - You're not gonna believe what just happened
2021-02-24T23:44:56.180Z <163>2 2021-02-24T23:44:56.180Z names.org devankoshal 8168 ID903 - Great Scott! We're never gonna reach 88 mph with the flux capacitor in its current state!
2021-02-24T23:44:56.190Z <58>2 2021-02-24T23:44:56.190Z we.net Karimmove 7105 ID300 - Great Scott! We're never gonna reach 88 mph with the flux capacitor in its current state!
2021-02-24T23:44:56.200Z <29>1 2021-02-24T23:44:56.200Z up.org shaneIxD 2747 ID774 - There's a breach in the warp core, captain
2021-02-24T23:44:56.210Z <106>2 2021-02-24T23:44:56.210Z for.com benefritz 8575 ID884 - We're gonna need a bigger boat
2021-02-24T23:44:56.221Z <83>1 2021-02-24T23:44:56.221Z some.org jesseddy 7273 ID869 - A bug was encountered but not in Vector, which

8. Regex Query
https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-regexp-query.html

In [66]:
query = {
  "query": {
    "regexp": {
      "message": {
        "value": "good*",
        "flags": "ALL",
        "max_determinized_states": 10000,
        "rewrite": "constant_score"
      }
    }
  }
}

res = client.search(index=indice, body=query)

print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(timestamp)s %(raw)s" % hit["_source"])

Got 204 Hits:
2021-02-24T23:44:56.160Z <117>2 2021-02-24T23:44:56.160Z random.net ahmadajmi 4895 ID3 - Pretty pretty pretty good
2021-02-24T23:44:56.250Z <43>1 2021-02-24T23:44:56.250Z up.org devankoshal 3940 ID420 - Pretty pretty pretty good
2021-02-24T23:45:55.335Z <106>1 2021-02-24T23:45:55.335Z make.net shaneIxD 5511 ID168 - Pretty pretty pretty good
2021-02-24T23:45:55.415Z <153>2 2021-02-24T23:45:55.415Z some.com shaneIxD 8837 ID385 - Pretty pretty pretty good
2021-02-24T23:45:55.475Z <109>1 2021-02-24T23:45:55.475Z we.de devankoshal 8977 ID671 - Pretty pretty pretty good
2021-02-24T23:45:55.515Z <154>2 2021-02-24T23:45:55.515Z for.com jesseddy 2257 ID395 - Pretty pretty pretty good
2021-02-24T23:45:55.645Z <17>1 2021-02-24T23:45:55.645Z some.com ahmadajmi 8640 ID872 - Pretty pretty pretty good
2021-02-24T23:45:55.705Z <162>1 2021-02-24T23:45:55.705Z up.com shaneIxD 9784 ID281 - Pretty pretty pretty good
2021-02-24T23:45:55.725Z <71>2 2021-02-24T23:45:55.725Z some.org meln1ks 954

9. Compount Query https://www.elastic.co/guide/en/elasticsearch/reference/current/compound-queries.html

In [74]:
query = {
   "query": {
      "bool" : {
         "must" : {
            "term" : { "hostname" : "random.net" }
         },
         "should": {
            "term" : { "application" : "ahmadajmi" }
         },
         "minimum_should_match" : 1,
         "boost" : 1.0
      }
   }
}


res = client.search(index=indice, body=query)

print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(timestamp)s %(raw)s" % hit["_source"])

Got 11 Hits:
2021-02-25T00:06:38.699Z <41>2 2021-02-25T00:06:38.699Z random.net ahmadajmi 502 ID324 - A bug was encountered but not in Vector, which doesn't have bugs
2021-02-25T00:06:41.319Z <90>2 2021-02-25T00:06:41.319Z random.net ahmadajmi 6424 ID358 - Maybe we just shouldn't use computers
2021-02-25T00:06:42.109Z <46>2 2021-02-25T00:06:42.109Z random.net ahmadajmi 7049 ID726 - Take a breath, let it go, walk away
2021-02-25T00:06:46.929Z <179>2 2021-02-25T00:06:46.929Z random.net ahmadajmi 1223 ID494 - We're gonna need a bigger boat
2021-02-24T23:44:56.160Z <117>2 2021-02-24T23:44:56.160Z random.net ahmadajmi 4895 ID3 - Pretty pretty pretty good
2021-02-24T23:45:55.975Z <128>1 2021-02-24T23:45:55.975Z random.net ahmadajmi 6363 ID167 - A bug was encountered but not in Vector, which doesn't have bugs
2021-02-24T23:45:57.335Z <0>2 2021-02-24T23:45:57.335Z random.net ahmadajmi 5148 ID875 - We're gonna need a bigger boat
2021-02-24T23:46:00.495Z <150>1 2021-02-24T23:46:00.495Z random.ne